In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship.git

In [ ]:
%cd DS_Internship/CV/src/

In [3]:
from utils import Sentinel2Downloader, load_torch_image, split_image_and_save
API_KEY = "/content/drive/MyDrive/Datasets/MATCHING/image-matching-key.json"

In [7]:
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0,
               'CLOUDY_PIXEL_PERCENTAGE': 10.0, }

loader = Sentinel2Downloader(api_key=API_KEY)

loaded = loader.download(product_type="L2A",
                        tiles=["36TXS"],
                        start_date="2020-02-01",
                        end_date="2020-12-30",
                        output_dir="/content/sentinel2imagery",
                        bands=set(["TCI"]),
                        cores=4,
                        constraints=CONSTRAINTS)

In [ ]:
import os

images_num = 0
image_names_dict = {}

for dirname, _, filenames in os.walk('/content/sentinel2imagery'):
    for filename in filenames:
        if filename.endswith('_TCI_10m.jp2'):
            images_num += 1
            if dirname not in image_names_dict:
                image_names_dict[dirname] = list()
            image_names_dict[dirname].append(filename)

print(images_num)
print(len([*image_names_dict]))

In [9]:
import os
val_filenames = ["/content/sentinel2imagery/S2B_MSIL2A_20200216T083929_N0214_R064_T36TXS_20200216T115323/T36TXS_20200216T083929_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2B_MSIL2A_20200824T083609_N0214_R064_T36TXS_20200824T112701/T36TXS_20200824T083609_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2A_MSIL2A_20200312T083731_N0214_R064_T36TXS_20200312T112715/T36TXS_20200312T083731_TCI_10m.jp2",
                 "/content/sentinel2imagery/S2A_MSIL2A_20200501T083601_N0214_R064_T36TXS_20200501T113014/T36TXS_20200501T083601_TCI_10m.jp2"]

crop_height = 480
crop_width = 640

dataset_dir = "/content/first_step_val_dataset"
os.makedirs(dataset_dir, exist_ok=True)

val_dir = os.path.join(dataset_dir, "val")

os.makedirs(val_dir, exist_ok=True)

for dirname, _, filenames in os.walk('/content/sentinel2imagery'):
    for filename in filenames:
        if filename.endswith('_TCI_10m.jp2'):
          basename = os.path.basename(filename)[:-4]
          tilepath = os.path.join(dirname, filename)

          if tilepath in val_filenames:
            save_path = os.path.join(val_dir, basename)

            image = load_torch_image(tilepath)

            split_image_and_save(
                image=image,
                crop_h=crop_height,
                crop_w=crop_width,
                save_crops=True,
                saving_path=save_path)




In [ ]:
import shutil
shutil.make_archive("/content/first_step_val_dataset", "zip", "/content/first_step_val_dataset/")